In [23]:
from mnist import MNIST 

mndata = MNIST('./data')

mndata.load_training()
img_side = 28
n_mnist_pixels = img_side*img_side

In [50]:
import numpy as np

layer_size = 10

I = np.zeros(n_mnist_pixels)
O = np.zeros(layer_size)
W = np.random.rand(O.size, I.size) * 0.001

In [51]:
push_delta = 0.4
k = 1

In [60]:
num_samples = 1
i_idx = 10

img = mndata.train_images[i_idx]

# infer
I = np.array(img) / 255
O = np.dot(W, I)

# train
dW = I - (W * O[np.newaxis].T)
wO = np.argsort(O)[::-1]
O.fill(0)
O[wO[0]] = 1
O[wO[1:1+k]] = -push_delta
dW *= O[np.newaxis].T
